In [1]:
%autosave 60

Autosaving every 60 seconds


## Проводим анализ существующих coding assistant

## Существующие решения 
- Stand alone приложения
- Плагины
- Чаты

## Уровни "помощи" со стороны ассистента
Что уже сейчас делают современные ассистенты

1. **Прямая генерация кода** (Быстрый ответ на локальный контекст) - Самая известная функция: вы начинаете писать код, и ассистент предлагает завершение строки или целый блок кода в виде "призрачного текста". Вы нажимаете Tab, и код появляется в файле. Это мгновенно и интуитивно.
2. **Интерактивный чат и генерация по запросу** - Вы можете "поговорить" с ассистентом в специальном окне чата. Выделяете код и спрашиваете: "Объясни, что здесь происходит", "Найди ошибку" или пишете комментарий //создай функцию для отправки email и получаете готовый код.
3. **Автоматический рефакторинг и "умные" правки** - Вы выделяете функцию, нажимаете "Рефакторинг с AI", и ассистент предлагает улучшенную версию кода (например, более читаемую или производительную). Изменения отображаются в виде diff-сравнения, и вы можете их принять или отклонить.
4. **Понимание всего проекта** (Full Codebase Awareness) - Ассистент дает ответы, учитывая весь ваш проект, а не только открытые файлы. Он знает о ваших кастомных классах, функциях-хелперах и общей архитектуре, что делает его подсказки невероятно точными.
5. **"Агенты" и автономные задачи** (Code Agents) - Вы ставите высокоуровневую задачу: "Добавь аутентификацию через GitHub" или "Создай новый API-эндпоинт для профиля пользователя". Агент сам составляет план, находит и редактирует нужные файлы, пишет код в нескольких местах и даже может запустить тесты для проверки. Он действует как младший разработчик.

## База под капотом каждого блока

### Инлайновые подсказки и автодополнение (Inline Autocompletion)
**Как это работает под капотом?** 

- Триггер: Срабатывает автоматически после паузы во время набора текста или принудительно по горячей клавише.
- Сбор контекста (локальный): Система собирает очень ограниченный, но релевантный контекст: текст до и после курсора в текущем файле, а также, возможно, пути к другим открытым файлам. Скорость здесь — главный приоритет.
- Формирование промпта: Контекст упаковывается в специальный промпт, часто с использованием техники Fill-in-the-Middle (FIM). Модели сообщают "начало" и "конец" кода, и ее задача — сгенерировать недостающую "середину".
- Запрос к модели: Запрос отправляется на сервер к легковесной, быстрой LLM, оптимизированной именно для генерации кода.
- Отображение: Полученный ответ мгновенно отображается в редакторе как неактивный текст.

In [4]:
# код примера для реализации Fill-in-the-Middle

### Интерактивный чат и генерация по запросу
**Как это работает под капотом?**

- Триггер: Явное действие пользователя — отправка сообщения в чате или выполнение команды.

- Сбор контекста (расширенный): Система собирает выделенный пользователем фрагмент кода, его вопрос на естественном языке и содержимое всего активного файла для лучшего понимания.

- Формирование промпта: Создается инструктивный промпт: "Пользователь выделил следующий код .... Он задал вопрос: .... Проанализируй и дай ответ".

- Запрос к модели: Запрос отправляется к более мощной, "разговорной" LLM (например, GPT-4), которая отлично справляется с анализом, объяснением и следованием инструкциям. Время ответа может быть больше, чем у автодополнения.

- Отображение: Ответ выводится в окне чата в виде текста, кода или их комбинации.

In [ ]:
# код пример, где мы сравниваем две LLM модели 
# - одна специальная для кодинга (coding)
# - вторая общая, не для кодинга

# и так же ссылаемся на бенчмарк кодинга

### Автоматический рефакторинг и "умные" правки

**Как это работает под капотом?**

- Анализ структуры кода: Помимо простого текста, система может парсить код в Абстрактное синтаксическое дерево (AST). Это позволяет AI "видеть" не просто символы, а структуру: переменные, функции, циклы.
- Формирование промпта: В промпт передается не только сам код, но и запрос на конкретное изменение: "Перепиши эту функцию, сделав ее более эффективной".
- Генерация и сравнение: LLM генерирует новую версию кода. Затем среда разработки (IDE) автоматически сравнивает старую и новую версии и создает diff-представление.
- Применение изменений: Если пользователь согласен, IDE заменяет старый код на новый.

In [ ]:
# код для построения ast дерева исходного кода
# и пример передачи зависимостей дерева в prompt

### Понимание всего проекта (Full Codebase Awareness)

**Как это работает под капотом?**

Это принципиально другой подход, основанный на Retrieval-Augmented Generation (RAG).

- **Индексация (заранее)**: При открытии проекта ассистент сканирует все файлы. Он разбивает код на логические части (функции, классы) и преобразует их в числовые представления — эмбеддинги — с помощью специальной модели.
- **Создание векторной базы**: Все эти эмбеддинги сохраняются в локальную или облачную векторную базу данных, которая позволяет осуществлять сверхбыстрый поиск по смысловой близости.
- **Поиск во время запроса**: Когда вы задаете вопрос, система превращает ваш запрос и текущий контекст в такой же эмбеддинг. Затем она ищет в векторной базе наиболее похожие (релевантные) фрагменты кода из всего проекта.
- **Обогащение промпта**: Найденные фрагменты кода добавляются в промпт для LLM как дополнительный контекст. Промпт выглядит так: "Вот вопрос пользователя .... Чтобы на него ответить, вот самый релевантный код из его проекта: .... Теперь дай ответ".

In [ ]:
# код с примером чанкинга - индексации кода
# - и поиском чанков (мини RAG Pipeline)

### "Агенты" и автономные задачи (Code Agents)

**Как это работает под капотом?**

Это циклический процесс, часто описываемый фреймворком ReAct (Reason + Act).

- **Планирование (Reason)**: LLM получает вашу цель и разбивает ее на последовательность конкретных шагов: "1. Найти файл с роутами. 2. Добавить новый роут. 3. Создать файл контроллера. 4. Написать логику в контроллере...".
- **Использование инструментов (Act)**: У "агента" есть доступ к набору инструментов — API для взаимодействия с IDE: readFile(path), writeFile(path, content), findSymbol(name), runTerminalCommand(command).
- **Исполнение и наблюдение**: Агент выполняет первый шаг плана, используя нужный инструмент (например, readFile('src/routes.js')). Он получает результат (содержимое файла или ошибку) и анализирует его.
- **Повторение цикла**: На основе результата агент корректирует свой план и переходит к следующему шагу. Этот цикл "Подумал -> Сделал -> Посмотрел на результат" продолжается до тех пор, пока задача не будет выполнена или агент не столкнется с проблемой, для решения которой ему нужна помощь человека.

In [ ]:
# подводка, что следующий ноутбук как раз про создание такого агента

In [ ]:
# конец